In [ ]:
%load_ext lab_black

In [1]:
import requests, json, random
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from config import weather_api_key
from citipy import citipy
from sklearn.linear_model import LinearRegression

In [13]:
# Build list of random coordinates

coordinate_list = []
counter = 0
number_of_points = 1200

while counter < number_of_points:
    lat = round(random.uniform(-90, 90), 6)
    lon = round(random.uniform(-180, 180), 6)
    counter += 1
    coordinate_list.append((lat, lon))

In [14]:
cities = []
city_count = 0

for coords in coordinate_list:
    city = citipy.nearest_city(coords[0], coords[1])
    if city.city_name not in cities:
        cities.append(city.city_name)
        city_count += 1

In [15]:
city_count

535

In [16]:
cities_results = []
count = 0
found_count = 0
not_found_count = 0

print("Beginning data retrieval:\n")
print(50 * "-")

for citi in cities:
    endpoint = "http://api.openweathermap.org/data/2.5/weather"
    payload = {"q": citi, "appid": weather_api_key, "units": "imperial"}

    r = requests.get(url=endpoint, params=payload)

    count += 1

    if r.status_code == 200:
        data = r.json()
        cities_results.append(data)
        print("Processing record: ", count, " | ", citi)
        found_count += 1
    else:
        print("Processing record: ", count, " | ", citi, " *Not Found*")
        not_found_count += 1

Beginning data retrieval:

--------------------------------------------------
Processing record:  1  |  dawlatabad
Processing record:  2  |  svetlogorsk
Processing record:  3  |  kapaa
Processing record:  4  |  barrow
Processing record:  5  |  vaini
Processing record:  6  |  punta arenas
Processing record:  7  |  cap malheureux
Processing record:  8  |  rikitea
Processing record:  9  |  port lincoln
Processing record:  10  |  bogoslovka
Processing record:  11  |  pevek
Processing record:  12  |  albany
Processing record:  13  |  ushuaia
Processing record:  14  |  homer
Processing record:  15  |  saldanha
Processing record:  16  |  lumeje
Processing record:  17  |  hamilton
Processing record:  18  |  nenjiang
Processing record:  19  |  barentsburg  *Not Found*
Processing record:  20  |  flinders
Processing record:  21  |  ontario
Processing record:  22  |  bredasdorp
Processing record:  23  |  castro
Processing record:  24  |  krasnyy chikoy
Processing record:  25  |  linxia
Processing 

In [17]:
r.status_code
# r.headers
# r.text
# r.json()

200

In [18]:
found_count

489

In [19]:
not_found_count

46

In [33]:
cities_results[0]
# 'weather' is a list of dicts, so not easily handled with pd.json_normalize()
# try manually unpacking as seen in class 9/17-1 at 35' mark

{'coord': {'lon': 64.91, 'lat': 36.41},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 66.67,
  'feels_like': 59.83,
  'temp_min': 66.67,
  'temp_max': 66.67,
  'pressure': 1020,
  'humidity': 24,
  'sea_level': 1020,
  'grnd_level': 972},
 'visibility': 10000,
 'wind': {'speed': 5.01, 'deg': 288},
 'clouds': {'all': 0},
 'dt': 1600837176,
 'sys': {'country': 'AF', 'sunrise': 1600824546, 'sunset': 1600868177},
 'timezone': 16200,
 'id': 1142226,
 'name': 'Dawlatabad',
 'cod': 200}

In [41]:
city_data = []

for city in cities_results:
    city_data.append({
        "City": city["name"],
        "Country": city["sys"]["country"],
        "Lat": city["coord"]["lat"],
        "Long": city["coord"]["lon"],
        "Temp_max": city["main"]["temp_max"],
        "Humidity": city["main"]["humidity"],
        "Cloudiness": city["clouds"]["all"],
        "Wind_Speed": city["wind"]["speed"]
    })

city_table = pd.DataFrame(city_data)

In [42]:
city_table

,City,Country,Lat,Long,Temp_max,Humidity,Cloudiness,Wind_Speed
0,Dawlatabad,AF,36.41,64.91,66.67,24,0,5.01
1,Forsytheganj,BY,52.63,29.73,53.38,66,0,6.42
2,Kapaa,US,22.08,-159.32,80.60,69,40,13.87
3,Barrow,US,71.29,-156.79,33.80,80,90,12.75
4,Vaini,TO,-21.20,-175.20,73.40,83,75,12.75
...,...,...,...,...,...,...,...,...
484,Moctezuma,MX,29.80,-109.67,78.03,45,26,3.00
485,Kutum,SD,14.20,24.67,77.76,59,50,2.68
486,Halifax,CA,44.65,-63.57,63.00,99,100,5.01
487,Tabou,CI,4.42,-7.35,75.09,90,97,8.19
